# 一份有关沪深300股票筛选的代码
日期范围是2005-4-8至2022-1-1

In [1]:
#导入包
import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 沪深300名单导入
hs300 = pd.read_excel(r"../data/沪深300指数成分股历年调整名单+更新至2022年1月1日.xlsx",dtype = object)
hs300_used = hs300.copy()
hs300_used

,成分股代码_CompoStkCd,成分股名称_CompoStkNm,开始日期_BegDt,结束日期_EndDt
0,000656,金科股份,2019-06-17,2021-12-10
1,000728,国元证券,2008-07-01,2021-12-10
2,000860,顺鑫农业,2020-06-15,2021-12-10
3,002153,石基信息,2014-12-15,2021-12-10
4,002384,东山精密,2020-12-14,2021-12-10
...,...,...,...,...
1088,688363,华熙生物,2021-06-15,NaT
1089,688396,华润微,2021-06-15,NaT
1090,688561,奇安信-U,2021-12-10,NaT
1091,688599,天合光能,2021-12-10,NaT


## 根据持续时间筛选

In [3]:
# 假设hs300_used是您的DataFrame，包含沪深300名单的信息
# 创建一个日期范围，用于筛选开始日期和结束日期之间的数据
start_date = pd.Timestamp('2005-04-08')
end_date = pd.Timestamp('2022-01-01')

# 使用条件筛选操作选择满足条件的数据
filtered_stock_list = hs300_used[(hs300_used['开始日期_BegDt'] >= start_date) & ((hs300_used['结束日期_EndDt'] <= end_date) | hs300_used['结束日期_EndDt'].isnull())]

# 将开始日期和结束日期转换为月份
filtered_stock_list['开始日期_BegDt'] = pd.to_datetime(filtered_stock_list['开始日期_BegDt']).dt.to_period('M')
filtered_stock_list['结束日期_EndDt'] = pd.to_datetime(filtered_stock_list['结束日期_EndDt']).dt.to_period('M')

# 将成分股代码和成分股名称转换为字符串类型
filtered_stock_list['成分股代码_CompoStkCd'] = filtered_stock_list['成分股代码_CompoStkCd'].astype(str)
filtered_stock_list['成分股名称_CompoStkNm'] = filtered_stock_list['成分股名称_CompoStkNm'].astype(str)

# 根据月份分组，筛选出每个月在名单内的股票
monthly_stock_list = filtered_stock_list.groupby(['开始日期_BegDt', '结束日期_EndDt']).agg({'成分股代码_CompoStkCd': lambda x: ', '.join(x), '成分股名称_CompoStkNm': lambda x: ', '.join(x)}).reset_index()

# 筛选掉结束日期在每个月份之前的股票
filtered_monthly_stock_list = pd.DataFrame(columns=['开始日期_BegDt', '结束日期_EndDt', '成分股代码_CompoStkCd', '成分股名称_CompoStkNm'])
for i, row in monthly_stock_list.iterrows():
    start_date = row['开始日期_BegDt']
    end_date = row['结束日期_EndDt']
    codes = row['成分股代码_CompoStkCd'].split(', ')
    stocks = row['成分股名称_CompoStkNm'].split(', ')
    filtered_codes = []
    filtered_stocks = []
    for code, stock in zip(codes, stocks):
        if end_date >= start_date or pd.isnull(end_date):
            filtered_codes.append(code)
            filtered_stocks.append(stock)
    filtered_monthly_stock_list = pd.concat([filtered_monthly_stock_list, pd.DataFrame({'开始日期_BegDt': [start_date], '结束日期_EndDt': [end_date], '成分股代码_CompoStkCd': [', '.join(filtered_codes)], '成分股名称_CompoStkNm': [', '.join(filtered_stocks)]})], ignore_index=True)

# 筛选结果
filtered_monthly_stock_list

,开始日期_BegDt,结束日期_EndDt,成分股代码_CompoStkCd,成分股名称_CompoStkNm
0,2005-04,2005-07,"000029, 000532, 000562, 000921, 600072, 600228...","深房集团, 华金资本, 宏源证券, 海信家电, 中船科技, 返利科技, 航天信息, 中金黄金..."
1,2005-04,2006-01,"000096, 000420, 000498, 000612, 000763, 000817...","广聚能源, 吉林化纤, 山东路桥, 焦作万方, 锦州石化, 辽河油田, 超声电子, 五矿稀土..."
2,2005-04,2006-04,"600002, 000406, 000866, 000956","齐鲁石化, 石油大明, 扬子石化, 中原油气"
3,2005-04,2006-07,"000016, 000429, 000507, 000511, 000533, 000543...","康佳集团, 粤高速, 珠海港, 烯碳新材, 顺钠股份, 皖能电力, 大洲控股, 粤宏远, 经..."
4,2005-04,2006-08,000780,平庄能源
...,...,...,...,...
329,2019-12,2020-12,"600928, 600968","西安银行, 海油发展"
330,2019-12,2021-06,"000723, 002958","美锦能源, 青农商行"
331,2020-06,2021-12,"000860, 002463, 601077","顺鑫农业, 沪电股份, 渝农商行"
332,2020-12,2021-12,"002384, 601872","东山精密, 招商轮船"


In [4]:
#导出
filtered_monthly_stock_list.to_csv('list1_持续时间.csv')

## 逐月更新名单

In [5]:
# 假设hs300_used是您的DataFrame，包含沪深300名单的信息
# 创建一个日期范围，包括从2005年4月开始到2021年12月结束的所有月份
date_range = pd.date_range(start='2005-04-01', end='2021-12-31', freq='M')
# 将日期范围转换为周期数据类型
months = date_range.to_period('M')
# 创建一个包含月份、股票代码和股票名称的DataFrame
monthly_stock_data = pd.DataFrame(columns=['month_details', 'stock', 'stock_name'])
# 遍历每个月份
for month in months:
    # 使用条件筛选操作选择满足条件的数据
    filtered_stock_list = hs300_used[(hs300_used['开始日期_BegDt'].dt.to_period('M') <= month) & 
                                     ((hs300_used['结束日期_EndDt'].dt.to_period('M') > month) | (hs300_used['结束日期_EndDt'].isnull()))]
    # 将开始日期和结束日期转换为月份
    filtered_stock_list['开始日期_BegDt'] = filtered_stock_list['开始日期_BegDt'].dt.to_period('M')
    filtered_stock_list['结束日期_EndDt'] = filtered_stock_list['结束日期_EndDt'].dt.to_period('M')
    # 将成分股代码和成分股名称转换为字符串类型
    filtered_stock_list['成分股代码_CompoStkCd'] = filtered_stock_list['成分股代码_CompoStkCd'].astype(str)
    filtered_stock_list['成分股名称_CompoStkNm'] = filtered_stock_list['成分股名称_CompoStkNm'].astype(str)
    # 遍历每个月份的筛选结果
    for index, row in filtered_stock_list.iterrows():
        start_date = row['开始日期_BegDt']
        end_date = row['结束日期_EndDt']
        codes = row['成分股代码_CompoStkCd'].split(', ')
        stocks = row['成分股名称_CompoStkNm'].split(', ')
        data = {'month_details': [month] * len(codes), 'stock': codes, 'stock_name': stocks}
        monthly_stock_data = pd.concat([monthly_stock_data, pd.DataFrame(data)])
monthly_stock_data

,month_details,stock,stock_name
0,2005-04,600177,雅戈尔
0,2005-04,000709,河钢股份
0,2005-04,600170,上海建工
0,2005-04,600188,兖州煤业
0,2005-04,600221,海航控股
...,...,...,...
0,2021-12,688363,华熙生物
0,2021-12,688396,华润微
0,2021-12,688561,奇安信-U
0,2021-12,688599,天合光能


In [6]:
# 月份赋值
month_name=monthly_stock_data['month_details'].unique()
month_number=range(85,286)
# 创建包含每个月份和相应数字的DataFrame
month_name_number = pd.DataFrame({'month_details': month_name, 'month':month_number})
# 合并dataframe
hc300_monthly = pd.merge(monthly_stock_data, month_name_number, on='month_details', how='left')
hc300_monthly.to_csv('hs300_monthly.csv')
hc300_monthly

,month_details,stock,stock_name,month
0,2005-04,600177,雅戈尔,85
1,2005-04,000709,河钢股份,85
2,2005-04,600170,上海建工,85
3,2005-04,600188,兖州煤业,85
4,2005-04,600221,海航控股,85
...,...,...,...,...
60293,2021-12,688363,华熙生物,285
60294,2021-12,688396,华润微,285
60295,2021-12,688561,奇安信-U,285
60296,2021-12,688599,天合光能,285


In [7]:
df=hc300_monthly.copy()
# 按照"month"进行分组，并将"stock"和"name"合并为列表
grouped_df = df.groupby("month").agg({"stock": list, "stock_name": list}).reset_index()
from ast import literal_eval
grouped_df ['stock'] = grouped_df ['stock'].apply(literal_eval)
grouped_df ['stock_name'] = grouped_df ['stock_name'].apply(literal_eval)
grouped_df['stock'] = grouped_df['stock'].apply(lambda x: list(set(x)))
grouped_df['stock_name'] = grouped_df['stock_name'].apply(lambda x: list(set(x)))
# 打印整合后的结果
grouped_df.to_csv('hs300.csv')
grouped_df

,month,stock,stock_name
0,85,"[600177, 000709, 600170, 600188, 600221, 60058...","[雅戈尔, 河钢股份, 上海建工, 兖州煤业, 海航控股, 海油工程, 铜陵有色, 同方股份..."
1,86,"[600177, 000709, 600170, 600188, 600221, 60058...","[雅戈尔, 河钢股份, 上海建工, 兖州煤业, 海航控股, 海油工程, 铜陵有色, 同方股份..."
2,87,"[600177, 000709, 600170, 600188, 600221, 60058...","[雅戈尔, 河钢股份, 上海建工, 兖州煤业, 海航控股, 海油工程, 铜陵有色, 同方股份..."
3,88,"[600177, 000709, 600170, 600188, 600221, 60058...","[雅戈尔, 河钢股份, 上海建工, 兖州煤业, 海航控股, 海油工程, 铜陵有色, 同方股份..."
4,89,"[600177, 000709, 600170, 600188, 600221, 60058...","[雅戈尔, 河钢股份, 上海建工, 兖州煤业, 海航控股, 海油工程, 铜陵有色, 同方股份..."
...,...,...,...
196,281,"[000656, 000728, 000860, 002153, 002384, 00245...","[金科股份, 国元证券, 顺鑫农业, 石基信息, 东山精密, 欧菲光, 沪电股份, 老板电器..."
197,282,"[000656, 000728, 000860, 002153, 002384, 00245...","[金科股份, 国元证券, 顺鑫农业, 石基信息, 东山精密, 欧菲光, 沪电股份, 老板电器..."
198,283,"[000656, 000728, 000860, 002153, 002384, 00245...","[金科股份, 国元证券, 顺鑫农业, 石基信息, 东山精密, 欧菲光, 沪电股份, 老板电器..."
199,284,"[000656, 000728, 000860, 002153, 002384, 00245...","[金科股份, 国元证券, 顺鑫农业, 石基信息, 东山精密, 欧菲光, 沪电股份, 老板电器..."
